#### Problem Statement: Predict the strength of the concrete

In [1]:
#pip install gradio

In [2]:
#### Import Libraries and Packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [3]:
df=pd.read_csv("concrete.csv")

In [4]:
df.head()

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
0,141.3,212.0,0.0,203.5,0.0,971.8,748.5,28,29.89
1,168.9,42.2,124.3,158.3,10.8,1080.8,796.2,14,23.51
2,250.0,0.0,95.7,187.4,5.5,956.9,861.2,28,29.22
3,266.0,114.0,0.0,228.0,0.0,932.0,670.0,28,45.85
4,154.8,183.4,0.0,193.3,9.1,1047.4,696.7,28,18.29


### Data Exploration->shape,mv,dtypes,info,data visualization

In [5]:
df.isnull().sum()

cement          0
slag            0
ash             0
water           0
superplastic    0
coarseagg       0
fineagg         0
age             0
strength        0
dtype: int64

#### Model Building->before

In [6]:
X=df.drop(["strength"],axis=1)
y=df[["strength"]]

In [7]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=1)

In [8]:
model_lr=LinearRegression()

In [9]:
model_lr.fit(X_train,y_train)

LinearRegression()

In [10]:
model_lr.score(X_train,y_train)

0.603148754063023

In [11]:
model_lr.score(X_test,y_test)

0.6339136715208276

In [12]:
#sns.pairplot(df)

In [13]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model

poly = PolynomialFeatures(degree=3, interaction_only=True)
X_train2 = poly.fit_transform(X_train)
X_test2 = poly.fit_transform(X_test)

poly_clf = linear_model.LinearRegression()

poly_clf.fit(X_train2, y_train)

#In sample (training) R^2 will always improve with the number of variables!
print(poly_clf.score(X_train2, y_train))

0.8161697487188945


In [14]:
print(poly_clf.score(X_test2, y_test))

0.7199719166739429


##### Approach1->Linear regression->60,63
##### Approach 2->Linear Regression +Removing outliers(without losing data)-> 72,73
##### Approach 3->Polynomial+degree 2->75,74
##### Approach 4->Polynomial+removing outliers(without losing data)->80,77

In [21]:
for idx, col_name in enumerate(X_train.columns):
    print("The coefficient for {} is {}".format(col_name, model_lr.coef_[0][idx]))

The coefficient for cement is 0.12655755592921444
The coefficient for slag is 0.1088284142362702
The coefficient for ash is 0.09070514083244793
The coefficient for water is -0.1031572400808996
The coefficient for superplastic is 0.37612343073244137
The coefficient for coarseagg is 0.024028594961397115
The coefficient for fineagg is 0.036192894560112224
The coefficient for age is 0.11043496734221792


In [18]:
intercept = model_lr.intercept_[0]
print("The intercept for our model is {}".format(intercept))

The intercept for our model is -52.96327115891326


In [ ]:
import gradio as gr
import pandas as pd
from sklearn.linear_model import LinearRegression

# Assuming model_lr is already trained somewhere before
# For the purpose of this example, I'll train a dummy model
# Replace this with your actual model training code
data = pd.DataFrame({
    "cement": [540.0, 540.0, 332.5, 332.5],
    "slag": [0.0, 0.0, 142.5, 142.5],
    "ash": [0.0, 0.0, 0.0, 0.0],
    "water": [162.0, 162.0, 228.0, 228.0],
    "superplastic": [2.5, 2.5, 0.0, 0.0],
    "coarseagg": [1040.0, 1055.0, 932.0, 932.0],
    "fineagg": [676.0, 676.0, 594.0, 594.0],
    "age": [28, 28, 270, 365],
    "strength": [79.99, 61.89, 40.27, 41.05]
})

X = data[["cement", "slag", "ash", "water", "superplastic", "coarseagg", "fineagg", "age"]]
y = data["strength"]

model_lr = LinearRegression()
model_lr.fit(X, y)

# Define a function to make predictions
def predict_strength(cement, slag, ash, water, superplastic, coarseagg, fineagg, age):
    data = pd.DataFrame({
        "cement": [cement],
        "slag": [slag],
        "ash": [ash],
        "water": [water],
        "superplastic": [superplastic],
        "coarseagg": [coarseagg],
        "fineagg": [fineagg],
        "age": [age]
    })
    prediction = model_lr.predict(data)[0]
    return prediction

# Define a function to print the equation of the model
def print_equation():
    coefficients = model_lr.coef_
    intercept = model_lr.intercept_
    equation = f"y = {intercept:.2f} "
    for i, coef in enumerate(coefficients):
        equation += f"+ ({coef:.2f} * x{i+1}) "
    return equation

# Create a Gradio interface
input_components = [
    gr.Number(label="Cement"),
    gr.Number(label="Slag"),
    gr.Number(label="Ash"),
    gr.Number(label="Water"),
    gr.Number(label="Superplastic"),
    gr.Number(label="Coarseagg"),
    gr.Number(label="Fineagg"),
    gr.Number(label="Age"),
]

output_component = gr.Textbox(label="Predicted Strength")
equation_output = gr.Textbox(label="Model Equation")

gr.Interface(
    fn=predict_strength,
    inputs=input_components,
    outputs=output_component,
    title="Concrete Strength Prediction"
).launch()

# Print the model equation separately
print(print_equation())


In [ ]:
import gradio as gr
import pandas as pd
from sklearn.linear_model import LinearRegression


# Define a function to make predictions
def predict_strength(cement, slag, ash, water, superplastic, coarseagg, fineagg, age):
    data = pd.DataFrame({
        "cement": [cement],
        "slag": [slag],
        "ash": [ash],
        "water": [water],
        "superplastic": [superplastic],
        "coarseagg": [coarseagg],
        "fineagg": [fineagg],
        "age": [age]
    })
    prediction = model_lr.predict(data)[0]
    return prediction

# Create a Gradio interface
input_components = [
    gr.Number(label="Cement"),
    gr.Number(label="Slag"),
    gr.Number(label="Ash"),
    gr.Number(label="Water"),
    gr.Number(label="Superplastic"),
    gr.Number(label="Coarseagg"),
    gr.Number(label="Fineagg"),
    gr.Number(label="Age"),
]

output_component = gr.Textbox(label="Predicted Strength")

gr.Interface(predict_strength, inputs=input_components, outputs=output_component, title="Concrete Strength Prediction").launch()


In [ ]:
0	141.3	212.0	0.0	203.5	0.0	971.8	748.5	28	29.89

In [ ]:
auto mpg->LR-> mpg and other attributes->80,84
LR after removing outliers->80,84
          PR before removing->
          PR after removing outliers->89,85